In [ ]:
import pandas as pd 
import numpy as np
import geopandas as gpd
import folium
from folium import Choropleth, LatLngPopup, Circle
from folium.plugins import HeatMap
from tqdm import tqdm

In [ ]:
# API DATA RETRIEVAL

import math
from time import sleep
from random import randint
import requests
from datetime import datetime
from dateutil.relativedelta import relativedelta
import calendar
import json
import operator

api_secrets = {
    'api_google_maps': ['key', '<PUT YOUR KEY HERE>']
}

In [ ]:
def get_gps_bounding_box(latitude, longitude, deg_lat, deg_lon, verbose):
    """
    Returns a bounding box derived from latitude/longitude deviated by
    deg_lat and deg_lon as GPS coordinates: n_lat, w_lon, s_lat, e_lon
    All coordinates in decimal degrees.

    Usage:
        n, w, s, e = get_gps_bounding_box(-88.77425, 37.05635, 1.0, 1.0, False)
        print(n, w, s, e)   # -87.77425 38.05635 -89.77425 36.05635

    """
    # lat1 < -90 or lat1 > 90   0 deg @ equator to +90 at N pole
    # lon1 < -180 or lon1 > 180     0 deg at Greenwich to +180 deg E

    import numpy

    if deg_lat < 0: raise Exception("ERROR: deg_lat passed to get_gps_bounding_box is invalid")
    if not isinstance(deg_lat, float): raise Exception("ERROR: deg_lat passed to get_gps_bounding_box is invalid")
    if deg_lon < 0: raise Exception("ERROR: deg_lon passed to get_gps_bounding_box is invalid")
    if not isinstance(deg_lon, float): raise Exception("ERROR: deg_lon passed to get_gps_bounding_box is invalid")

    if latitude >= 0:
        if latitude + deg_lat >= 90.0:
            n = 90.0 * numpy.sign(latitude)
            s = latitude - deg_lat
        else:
            n = latitude + deg_lat
            s = latitude - deg_lat

    else:
        # latitude < 0
        if abs(latitude) + deg_lat >= 90.0:
            n = 90.0 * numpy.sign(latitude)
            s = (abs(latitude) - deg_lat) * numpy.sign(latitude)
        else:
            n = latitude + deg_lat
            s = latitude - deg_lat

    if longitude >= 0:
        if longitude + deg_lon >= 180.0:
            w = 180.0 * numpy.sign(longitude)
            e = longitude - deg_lon
        else:
            w = longitude + deg_lon
            e = longitude - deg_lon

    else:
        # longitude < 0
        if abs(longitude) + deg_lon >= 180.0:
            w = 180.0 * numpy.sign(longitude)
            e = (abs(longitude) - deg_lon) * numpy.sign(longitude)
        else:
            w = longitude + deg_lon
            e = longitude - deg_lon

    if verbose: print(round(n,1), round(latitude,1), round(s,1), "\t", round(w,1), round(longitude,1), round(e,1))

    return n, w, s, e

In [ ]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Returns the distance in meters between the two GPS coordinates in decimal degrees.

    Usage:

        dist_m = haversine_distance(40.440363, -76.126746, 40.440406, -76.121293)
        print("dist_m: ", dist_m)
        # Result is 461.5 m.  The correct result is 461.5 m.

    """
    # Radius of the Earth in meters
    earth_radius_m = 6371000.0

    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Differences between the latitudes and longitudes
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Calculate the distance
    distance = earth_radius_m * c

    return distance

In [ ]:
def elevation_by_lat_lon(lat1, lon1):
    """
    Returns the elevation in meters for the specified latitude/longitude, or None
    if the service could not determine the elevation.

    Uses Google Maps API for elevation, or Open-Elevation if no result.

    Open-Elevation
    website: https://open-elevation.com/
    docs: https://github.com/Jorl17/open-elevation/blob/master/docs/api.md
    endpoint: curl 'https://api.open-elevation.com/api/v1/lookup?locations=10,10|20,20|41.161758,-8.583933'

    Other API alternatives:
        TessaDEM (https://tessadem.com/) has a free API.
        GPXZ API is free at 100 requests per day, otherwised $49/mo.  https://www.gpxz.io/#pricing

    Usage:
        elevation_m = elevation_by_lat_lon(38.5288, -78.4383)
        print('Elevation for 38.5288, -78.4383: ', elevation_m, 'meters')

    """

    if not "api_google_maps" in api_secrets.keys(): raise Exception("ERROR: api_secrets doesn't have the key requested for 'api_google_maps'.")

    # Try using Google Maps API for elevation
    url = "https://maps.googleapis.com/maps/api/elevation/json?locations=" + str(lat1) + "," + str(lon1) + "&" + api_secrets['api_google_maps'][0] + "=" + api_secrets['api_google_maps'][1] + ""
    resp = None
    #sleep(randint(1,3))     # random pause between 1 and 3 seconds
    try:
        resp = requests.get(url).json()
    except Exception as e:
        print('ERROR: ' + repr(e), url)

    if resp is None:
        print('maps.googleapis.com failed.  Trying api.open-elevation.com..')
        url = 'https://api.open-elevation.com/api/v1/lookup?locations=' + str(lat1) + ',' + str(lon1)
        sleep(randint(2,5))     # random pause between 2 and 5 seconds
        try:
            resp = requests.get(url).json()
        except Exception as e:
            print('ERROR: ' + repr(e), url)

    #print(json.dumps(data, sort_keys=True, indent=2))
    if not 'results' in resp.keys():
        print(json.dumps(resp, sort_keys=True, indent=2))
        return None

    results = resp['results'][0]
    if not 'elevation' in results.keys(): return None
    elevation = float(results['elevation'])
    return elevation

In [ ]:
def get_stations_by_bounding_box(lat, lon, verbose):
    """
    Using the latest API endpoint (as of 2023)

    Returns a list of stations along with their GPS coordinates and start/end dates,
    and the computed distance and elevation delta from the target location specified
    by lat,lon that include the dataTypes=TMIN,TMAX,AWND.

    Usage:

    stations = get_stations_by_bounding_box(40.44077631612291, -76.12267163754181, False)
    if stations is None:
        print("NO results")
    else:
        print("The best station choice is: ", stations[0][3], "from ", stations[0][6], " to ", stations[0][7])
        # All data in stations:
        print("sort, NO ELEVATION, delta_distance_m, ID, latitude, longitude, dateStart, dateEnd")
        for station in stations:
            print(station)

    Output:

    The best station choice is:  USW00013722 from  1944-06-01T00:00:00  to  2023-09-30T23:59:59
    sort, NO ELEVATION, delta_distance_m, ID, latitude, longitude, dateStart, dateEnd
    [8818330.2, 15.8, 556567.6, 'USW00013722', 35.89227, -78.78194, '1944-06-01T00:00:00', '2023-09-30T23:59:59']
    [12044151.2, 83.7, 143881.9, 'USC00286055', 40.47282, -74.42259, '1968-06-01T00:00:00', '2023-08-31T23:59:59']
    [31981786.2, 113.8, 281154.6, 'USC00303184', 42.8766, -77.0307, '1969-01-01T00:00:00', '2023-08-31T23:59:59']
    [60606995.9, 67.5, 897346.3, 'USC00177238', 45.08533, -67.1205, '1994-11-01T00:00:00', '2023-08-31T23:59:59']
    [100554773.3, 107.7, 933870.5, 'USC00122309', 38.4558, -86.6983, '1955-01-01T00:00:00', '2023-07-31T23:59:59']
    [110337972.5, 120.7, 913908.0, 'USC00129222', 41.4439, -86.9294, '1961-01-01T00:00:00', '2023-08-31T23:59:59']
    [280852969.2, 189.9, 1479068.0, 'USC00218450', 44.9902, -93.17995, '1960-10-01T00:00:00', '2023-08-31T23:59:59']
    [430710417.9, 251.6, 1711708.5, 'USC00255362', 41.143, -96.4808, '1968-11-01T00:00:00', '2023-08-31T23:59:59']
    [1233513339.3, 579.1, 2130165.0, 'USC00395544', 44.5171, -101.6184, '1911-10-01T00:00:00', '2023-08-31T23:59:59']
    [3908089408.1, 1356.5, 2880981.5, 'USC00241047', 45.6729, -111.1547, '1966-11-01T00:00:00', '2023-08-31T23:59:59']

    """

    gps_n, gps_w, gps_s, gps_e = get_gps_bounding_box(lat, lon, 1.0, 1.0, verbose)

    #url = "https://www.ncei.noaa.gov/access/services/search/v1/data?dataset=global-summary-of-the-month&bbox=35.462327,-82.563951,35.412327,-82.513951&dataTypes=TMIN,TMAX,PRCP&limit=10&offset=0"
    url = "https://www.ncei.noaa.gov/access/services/search/v1/data?dataset=global-summary-of-the-month"
    url += "&bbox=" + str(gps_n) + "," + str(gps_w) + "," + str(gps_s) + "," + str(gps_e)
    url += "&dataTypes=TMIN,TMAX,AWND"
    url += "&limit=1&offset=0"
    if verbose: print(url)

    headers = {'token': '<PUT YOUR TOKEN HERE>'}

    #sleep(randint(1,3))     # random pause between 1 and 3 seconds
    try:
        req = requests.get(url, data=None, json=None, headers=None)
    except Exception as e:
        print('ERROR: ' + repr(e), ' fn()', url)
        return None

    if not req.status_code == 200:
        if req.status_code == 429:
            raise Exception('ERROR: HTTP 429 Too Many Requests! rate limiting.  resp.status_code = ', str(req.status_code), req.text)
        else:
            print('\tERROR: resp.status_code = ', str(req.status_code), req.text)
            return None

    resp = req.json()
    if len(resp) == 0:
        print("\tNo results from query ", url)
        return None

    #print(json.dumps(resp, sort_keys=False, indent=2))
    station_data = []
    #if 'data' in resp.keys():
    if not 'results' in resp.keys(): raise Exception("ERROR: key 'results' not in response")
    results = resp['results']
    for result in results:
        if not 'endDate' in result.keys(): raise Exception("ERROR: key 'endDate' not in response (result)")
        if not 'startDate' in result.keys(): raise Exception("ERROR: key 'startDate' not in response (result)")
        if not 'centroid' in result.keys(): raise Exception("ERROR: key 'centroid' not in response (result)")
        if not 'point' in result['centroid'].keys(): raise Exception("ERROR: key 'point' not in response (result['centroid'])")
        centroid = result['centroid']
        longitude = float(centroid['point'][0])
        latitude = float(centroid['point'][1])
        if not 'name' in result.keys(): raise Exception("ERROR: key 'name' not in response (result)")
        if not 'location' in result.keys(): raise Exception("ERROR: key 'location' not in response (result)")
        if not 'id' in result.keys(): raise Exception("ERROR: key 'id' not in response (result)")
        if not 'dataTypesCount' in result.keys(): raise Exception("ERROR: key 'dataTypesCount' not in response (result)")
        if not 'boundingPoints' in result.keys(): raise Exception("ERROR: key 'boundingPoints' not in response (result)")
        if not 'stations' in result.keys(): raise Exception("ERROR: key 'stations' not in response (result)")
        stations = result['stations']
        for station in stations:
            if not 'name' in station.keys(): raise Exception("ERROR: key 'name' not in response (stations)")
            if not 'id' in station.keys(): raise Exception("ERROR: key 'id' not in response (stations)")
            delta_distance_m = haversine_distance(lat, lon, latitude, longitude)
            sort = delta_distance_m
            if verbose: print(station['id'], "\t", latitude, longitude, "\t", round(delta_distance_m,1), "NO ELEVATION AVALIABLE", result['startDate'], "\t", result['endDate'])
            station_data.append([round(sort,1), "NO ELEVATION AVALIABLE", round(delta_distance_m,1), station['id'], latitude, longitude, result['startDate'], result['endDate']])

    # Sort the data by delta_distance_m
    station_data.sort(reverse=False, key=operator.itemgetter(0))
    return station_data

In [ ]:
def get_mly_climate_data_for_station(stationid, startdate, enddate, target_year, verbose):
    """
    Using the station data from get_stations_by_bounding_box(), query using the new API
    and obtain the monthly climate data (TMIN,TMAX,PRCP) for the last full year between
    startdate and enddate.

    Example:

    # Using the station data obtained from get_stations_by_bounding_box()
    station = [8818330.2, 15.8, 556567.6, 'USW00013722', 35.89227, -78.78194, '1944-06-01T00:00:00', '2023-09-30T23:59:59']
    print("\nStation ID: ", station[3])
    print("start date: ", station[6])
    print("end date: ", station[7])
    print("\n")
    # get_mly_climate_data_for_station(stationid, startdate, enddate, verbose):
    mly_climate_data = get_mly_climate_data_for_station(station[3], station[6], station[7], False)
    if mly_climate_data is None:
        print("ERROR: No monthly climate data available for " + station[3] + " but it was expected!")
    else:
        print("Month\tTmin[F]\tTmax[F]\tPRCP[in]")
        for m in mly_climate_data:
            print(m[0], '\t', m[1], '\t', m[2], '\t', m[3])


    """

    # 'GHCND:USC00020060' -> 'USC00020060'
    if str(stationid).count(':') == 1:
        stationid = str(stationid).partition(":")[2]    # 'GHCND:USC00020060' -> 'USC00020060'
        print("stationid: '" + stationid + "'")

    # Convert date string to a datetime object
    if str(startdate).count('T') > 0:
        startdate = str(startdate).partition("T")[0]
        start = datetime.strptime(startdate, "%Y-%m-%d")
    else:
        # strptime(date_str, format)
        start = datetime.strptime(startdate, "%Y-%m-%d")

    if str(enddate).count('T') > 0:
        enddate = str(enddate).partition("T")[0]
        end = datetime.strptime(enddate, "%Y-%m-%d")
    else:
        # strptime(date_str, format)
        end = datetime.strptime(enddate, "%Y-%m-%d")

    if start > end: raise Exception("ERROR: date " + str(start) + " comes after " + str(end))
    if target_year > end.year - 1: raise Exception("ERROR: target_year " + str(target_year) + " comes after " + str(end.year - 1))
    if target_year < start.year: raise Exception("ERROR: target_year " + str(target_year) + " comes before " + str(start.year))

    startdate = datetime.strftime(datetime(target_year, 1, 1, 12, 1, 1), "%Y-%m-%d")
    enddate = datetime.strftime(datetime(target_year, 12, 1, 12, 1, 1), "%Y-%m-%d")

    # New Endpoint
    #      https://www.ncei.noaa.gov/access/services/data/v1?dataset=global-summary-of-the-month&dataTypes=TMIN,TMAX,PRCP&stations=USC00286055&startDate=2022-01-01&endDate=2022-12-31&format=json&units=standard&includeAttributes=false
    url = "https://www.ncei.noaa.gov/access/services/data/v1?dataset=global-summary-of-the-month&dataTypes=TMIN,TMAX,AWND&stations=" + stationid + "&startDate=" + startdate + "&endDate=" + enddate + "&format=json&units=standard&includeAttributes=false"
    if verbose: print(url)

    headers = {'Sec-Ch-Ua-Platform': 'Windows',
              'Sec-Ch-Ua-Platform-Version': '10.0.0',
              'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}

    # *** Update below with your NOAA token
    headers.update({"accept-language": "en-US,en;q=0.9", 'token': 'PUT YOUR TOKEN HERE'})

    try:
        req = requests.get(url, data=None, json=None, headers=headers)
    except Exception as e:
        print('ERROR: ' + repr(e), ' get_mly_climate_data_for_station()', url)
        return None

    if req.status_code == 200:
        pass
    elif req.status_code == 429:
        print("ERROR: HTTP 429 Too Many Requests! rate limiting.  resp.status_code = ", str(req.status_code), req.text, " get_mly_climate_data_for_station()")
        return None
    elif req.status_code == 503:
        print("ERROR: HTTP 503 Service Unavailable.  Server is temporarily unavailable due to maintenance or capacity issues.  get_mly_climate_data_for_station()")
        return None
    else:
        print('\tERROR: resp.status_code = ', str(req.status_code), req.text, "get_mly_climate_data_for_station()")
        return None

    resp = req.json()
    if len(resp) == 0:
        if verbose: print("\tNo results from query ", url, "\get_mly_climate_data_for_station()")
        return None
    elif len(resp) < 12:
        if verbose: print("\tLess than 12 months of data available.  ", url, "\get_mly_climate_data_for_station()")
        return None
    else:
        pass

    #print(json.dumps(resp, sort_keys=False, indent=2))

    # Make sure the data has TMIN,TMAX,AWND
    all_data_exists = True
    for m in resp:
        #print(m)        # {'DATE': '2022-01', 'STATION': 'USC00286055', 'TMAX': '37.7', 'TMIN': '20.9', 'PRCP': '4.56'}
        if not "DATE" in m.keys(): raise Exception("ERROR: key 'DATE' expected but not found. ", m)
        if not "STATION" in m.keys(): raise Exception("ERROR: key 'STATION' expected but not found. ", m)
        if not "TMAX" in m.keys(): all_data_exists = False
        if not "TMIN" in m.keys(): all_data_exists = False
        if not "AWND" in m.keys(): all_data_exists = False
    if not all_data_exists:
        print("ERROR: The response doesn't have all of the data for TMIN,TMAX,AWND.")
        return None

    mly_data = []
    metadata = {}
    n = 0
    for m in resp:
        n += 1
        month_date = datetime.strptime(m['DATE'], "%Y-%m")
        mly_data.append([calendar.month_abbr[n], m['TMIN'], m['TMAX'], m['AWND']])

    return mly_data

In [ ]:
from datetime import datetime, timedelta

def get_dly_climate_data_for_station(stationid, startdate, enddate, targetdate, verbose):

  if str(stationid).count(':') == 1:
      stationid = str(stationid).partition(":")[2]    # 'GHCND:USC00020060' -> 'USC00020060'
      print("stationid: '" + stationid + "'")

  # Convert date string to a datetime object
  if str(startdate).count('T') > 0:
      startdate = str(startdate).partition("T")[0]
      start = datetime.strptime(startdate, "%Y-%m-%d")
  else:
      # strptime(date_str, format)
      start = datetime.strptime(startdate, "%Y-%m-%d")

  if str(enddate).count('T') > 0:
      enddate = str(enddate).partition("T")[0]
      end = datetime.strptime(enddate, "%Y-%m-%d")
  else:
      # strptime(date_str, format)
      end = datetime.strptime(enddate, "%Y-%m-%d")

  if str(startdate).count('T') > 0:
      startdate = str(startdate).partition("T")[0]
      start = datetime.strptime(startdate, "%Y-%m-%d")
  else:
      # strptime(date_str, format)
      start = datetime.strptime(startdate, "%Y-%m-%d")

  if str(targetdate).count('T') > 0:
      targetdate = str(targetdate).partition("T")[0]
      target = datetime.strptime(targetdate, "%Y-%m-%d")
  else:
      # strptime(date_str, format)
      target = datetime.strptime(targetdate, "%Y-%m-%d")

  if start > end: 
    print('raise Exception("ERROR: date " + str(start) + " comes after " + str(end))') # raise Exception("ERROR: date " + str(start) + " comes after " + str(end))
    return None
  if target.year > end.year - 1: 
    print('raise Exception("ERROR: target_year " + str(target_year) + " comes after " + str(end.year - 1))') # raise Exception("ERROR: target_year " + str(target_year) + " comes after " + str(end.year - 1))
    return None 
  if target.year < start.year + 1: 
    print('raise Exception("ERROR: target_year " + str(target_year) + " comes before " + str(start.year + 1))') # raise Exception("ERROR: target_year " + str(target_year) + " comes before " + str(start.year + 1))
    return None

  start_of_week = target - timedelta(days=target.weekday())
  start_of_week = start_of_week.replace(hour=0, minute=0, second=0, microsecond=0)

  # Calculate the end of the week (Sunday 23:59:59.999999)
  end_of_week = start_of_week + timedelta(days=6, hours=23, minutes=59, seconds=59, microseconds=999999)

  startdate = datetime.strftime(start_of_week, "%Y-%m-%d")
  enddate = datetime.strftime(end_of_week, "%Y-%m-%d")

  url = "https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&dataTypes=TMIN,TMAX,AWND&stations=" + stationid + "&startDate=" + startdate + "&endDate=" + enddate + "&format=json&units=standard&includeAttributes=false"
  if verbose: print(url)

  headers = {'Sec-Ch-Ua-Platform': 'Windows',
            'Sec-Ch-Ua-Platform-Version': '10.0.0',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}

  # *** Update below with your NOAA token
  headers.update({"accept-language": "en-US,en;q=0.9", 'token': '<PUT YOUR TOKEN HERE'})

  try:
      req = requests.get(url, data=None, json=None, headers=headers)
  except Exception as e:
      print('ERROR: ' + repr(e), ' get_mly_climate_data_for_station()', url)
      return None

  if req.status_code == 200:
        pass
  elif req.status_code == 429:
      print("ERROR: HTTP 429 Too Many Requests! rate limiting.  resp.status_code = ", str(req.status_code), req.text, " get_mly_climate_data_for_station()")
      return None
  elif req.status_code == 503:
      print("ERROR: HTTP 503 Service Unavailable.  Server is temporarily unavailable due to maintenance or capacity issues.  get_mly_climate_data_for_station()")
      return None
  else:
      print('\tERROR: resp.status_code = ', str(req.status_code), req.text, "get_mly_climate_data_for_station()")
      return None

  resp = req.json()
  if len(resp) == 0:
      if verbose: print("\tNo results from query ", url, "\get_mly_climate_data_for_station()")
      return None
  elif len(resp) < 7:
      if verbose: print("\tLess than 7 days of data available.  ", url, "\get_dly_climate_data_for_station()")
      return None
  else:
      pass


  # Make sure the data has TMIN,TMAX,AWND
  all_data_exists = True
  for m in resp:
      #print(m)        # {'DATE': '2022-01', 'STATION': 'USC00286055', 'TMAX': '37.7', 'TMIN': '20.9', 'PRCP': '4.56'}
      if not "DATE" in m.keys(): raise Exception("ERROR: key 'DATE' expected but not found. ", m)
      if not "STATION" in m.keys(): raise Exception("ERROR: key 'STATION' expected but not found. ", m)
      if not "TMAX" in m.keys(): all_data_exists = False
      if not "TMIN" in m.keys(): all_data_exists = False
      if not "AWND" in m.keys(): all_data_exists = False
  if not all_data_exists:
      print("ERROR: The response doesn't have all of the data for TMIN,TMAX,AWND.")
      return None

  dly_data = []
  for d in resp:
    ddate = datetime.strptime(d['DATE'], "%Y-%m-%d")
    dly_data.append([ddate, d['TMIN'], d['TMAX'], d['AWND']])

  return dly_data



In [ ]:
# TESTING OUTPUT FOR RETRIEVING MONTHLY DATA

cur_lat = 37.338207
cur_long = -121.886330
verbose_flag = 0

target_year = 2022

stations = get_stations_by_bounding_box(cur_lat, cur_long, verbose_flag)
if stations and len(stations) > 0:
  station = stations[0]
  print("station id: ", str(station[3]))
  print("start date: ", station[6])
  print("end date: ", station[7])
  mly_climate_data = get_mly_climate_data_for_station(station[3], station[6], station[7], target_year, False);
  if mly_climate_data is None:
    print("ERROR: No monthly climate data available for " + station[3] + " but it was expected!")
  else:
    print("Month\tTmin[F]\tTmax[F]\tAWND[mi/hr]")
    for m in mly_climate_data:
      print(m[0], '\t', m[1], '\t', m[2], '\t', m[3])
else:
  print("ERROR: no stations near by")

In [ ]:
# TESTING OUTPUT FOR RETRIEVING DAILY DATA

cur_lat = 37.363796
cur_long = -121.91234
verbose_flag = 0
target_date = "2010-01-01"

stations = get_stations_by_bounding_box(cur_lat, cur_long, verbose_flag)
if stations and len(stations) > 0:
  station = stations[0]
  print("station id: ", str(station[3]))
  print("start date: ", station[6])
  print("end date: ", station[7])
  print("target date: ", target_date)
  dly_climate_data = get_dly_climate_data_for_station(station[3], station[6], station[7], target_date, False);
  if dly_climate_data is None:
    print("ERROR: No daily climate data available for " + station[3] + " but it was expected!")
  else:
    print("Date[YYYY-MM-DD]\tTmin[F]\tTmax[F]\tAWND[mi/hr]")
    for d in dly_climate_data:
      print(d[0].date(), '\t\t', d[1], '\t', d[2], '\t', d[3])
else:
  print("ERROR: no stations near by")

In [ ]:
# DISCLAIMER: THE WEBSCRAPING CODE USED ABOVE WAS TAKEN FROM https://medium.com/@markwkiehl/noaa-climate-api-c2e36e7a49c5